### Given a topic research the latest news

In [1]:
import sys

sys.path.append("..")

from dotenv import load_dotenv

load_dotenv()

from utils import utils

In [2]:
utils.SCRIPT = "outputs/robertson/script.yaml"
scenes = utils.get_scenes()
# Extract lines starting with "[Scene"
scene_lines = [s.description for s in scenes]
scene_lines

['A somber set, with a backdrop of religious symbols and the iconic image of Pat Robertson. The screen briefly flashes with the date: June 8, 2023.',
 "A montage of Pat Robertson's early life, with images of his birthplace, his family, his time in the Marines, and his academic life.",
 'Visuals of a younger Pat Robertson with his wife, images of a New York commune, and his first steps into religious life.',
 "Clips of the Christian Broadcasting Network, its shows, and a glimpse of the '700 Club'.",
 "Visuals of Robertson's presidential campaign, his involvement with the Christian Coalition, and his influence on conservative politics.",
 "Images of Regent University, the American Center for Law and Justice, Operation Blessing, and clips of Robertson's controversial statements.",
 "The scene shows the YouTube channel's logo and animations.",
 'A return to the somber set from the first scene, this time featuring a photo of Pat Robertson in his later years.']

In [3]:
import torch
from diffusers import StableDiffusionPipeline

pipeline = StableDiffusionPipeline.from_pretrained(
    "SG161222/Realistic_Vision_V2.0", torch_dtype=torch.float16
)

vae/diffusion_pytorch_model.safetensors not found
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [4]:
pipeline = pipeline.to("cuda")
pipeline.enable_xformers_memory_efficient_attention()

In [5]:
pipeline.scheduler.compatibles

[diffusers.schedulers.scheduling_dpmsolver_singlestep.DPMSolverSinglestepScheduler,
 diffusers.schedulers.scheduling_heun_discrete.HeunDiscreteScheduler,
 diffusers.schedulers.scheduling_euler_ancestral_discrete.EulerAncestralDiscreteScheduler,
 diffusers.schedulers.scheduling_unipc_multistep.UniPCMultistepScheduler,
 diffusers.schedulers.scheduling_dpmsolver_multistep.DPMSolverMultistepScheduler,
 diffusers.schedulers.scheduling_ddpm.DDPMScheduler,
 diffusers.schedulers.scheduling_pndm.PNDMScheduler,
 diffusers.schedulers.scheduling_euler_discrete.EulerDiscreteScheduler,
 diffusers.schedulers.scheduling_deis_multistep.DEISMultistepScheduler,
 diffusers.utils.dummy_torch_and_torchsde_objects.DPMSolverSDEScheduler,
 diffusers.schedulers.scheduling_lms_discrete.LMSDiscreteScheduler,
 diffusers.schedulers.scheduling_k_dpm_2_ancestral_discrete.KDPM2AncestralDiscreteScheduler,
 diffusers.schedulers.scheduling_ddim.DDIMScheduler,
 diffusers.schedulers.scheduling_k_dpm_2_discrete.KDPM2Discret

In [6]:
from diffusers import DPMSolverMultistepScheduler

pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)

In [7]:
positive_prompt = """
(high detailed skin:1.2, high quality, reportage, journalism),
8k uhd, dslr, soft lighting, film grain, Fujifilm XT3
"""

negative_prompt = """
((text:1.5, deformed iris, deformed pupils, deformed hands, nsfw)),
((semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4)),
close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate,
morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation,
deformed, blurry, burns, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured,
gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs,
fused fingers, too many fingers, long neck
"""

In [9]:
guidance_scale = 0.7
num_inference_steps = 50
num_images_per_prompt = 5
image_height = 512
image_width = 768

for i, line in enumerate(scene_lines):
    prompt = line
    images = pipeline(
        prompt="{} {}".format(prompt, positive_prompt),
        negative_prompt=negative_prompt,
        height=image_height,
        width=image_width,
        num_images_per_prompt=num_images_per_prompt,
        num_inference_steps=num_inference_steps,
        # guidance_scale=guidance_scale,
    ).images

    for j, image in enumerate(images):
        image.save("outputs/robertson/scene_{}_{}.png".format(i + 1, j + 1))

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [12]:
# Free GPU memory for upscaler
del pipeline
del images
torch.cuda.empty_cache()

In [1]:
from PIL import Image
from diffusers import StableDiffusionUpscalePipeline

import torch

pipeline = StableDiffusionUpscalePipeline.from_pretrained(
    "stabilityai/stable-diffusion-x4-upscaler",
    revision="fp16",
    torch_dtype=torch.float16,
)

text_encoder/model.safetensors not found


In [2]:
from xformers.ops import MemoryEfficientAttentionFlashAttentionOp

pipeline = pipeline.to("cuda")
pipeline.enable_xformers_memory_efficient_attention(
    attention_op=MemoryEfficientAttentionFlashAttentionOp
)
# Workaround for not accepting attention shape using VAE for Flash Attention
pipeline.vae.enable_xformers_memory_efficient_attention(attention_op=None)

In [3]:
pipeline.scheduler.compatibles

[diffusers.schedulers.scheduling_k_dpm_2_discrete.KDPM2DiscreteScheduler,
 diffusers.schedulers.scheduling_dpmsolver_multistep.DPMSolverMultistepScheduler,
 diffusers.schedulers.scheduling_ddim.DDIMScheduler,
 diffusers.schedulers.scheduling_lms_discrete.LMSDiscreteScheduler,
 diffusers.schedulers.scheduling_ddpm.DDPMScheduler,
 diffusers.schedulers.scheduling_pndm.PNDMScheduler,
 diffusers.schedulers.scheduling_dpmsolver_singlestep.DPMSolverSinglestepScheduler,
 diffusers.schedulers.scheduling_deis_multistep.DEISMultistepScheduler,
 diffusers.schedulers.scheduling_unipc_multistep.UniPCMultistepScheduler,
 diffusers.utils.dummy_torch_and_torchsde_objects.DPMSolverSDEScheduler,
 diffusers.schedulers.scheduling_euler_discrete.EulerDiscreteScheduler,
 diffusers.schedulers.scheduling_k_dpm_2_ancestral_discrete.KDPM2AncestralDiscreteScheduler,
 diffusers.schedulers.scheduling_euler_ancestral_discrete.EulerAncestralDiscreteScheduler,
 diffusers.schedulers.scheduling_heun_discrete.HeunDiscret

In [4]:
from diffusers import DPMSolverMultistepScheduler

pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)

In [12]:
guidance_scale = 0.7
num_inference_steps = 20
num_images_per_prompt = 1
n = 5

for i, prompt in enumerate(scene_lines):
    for j in range(1, n + 1):
        low_res_image = Image.open(
            "outputs/robertson/scene_{}_{}.png".format(i + 1, j)
        ).convert("RGB")
        low_res_image = low_res_image.resize((256, 256))

        image = pipeline(
            prompt="{} {}".format(prompt, positive_prompt),
            negative_prompt=negative_prompt,
            image=low_res_image,
            num_images_per_prompt=num_images_per_prompt,
            num_inference_steps=num_inference_steps,
            # guidance_scale=guidance_scale,
        ).images[0]

        image.save("outputs/robertson/scene_{}_{}_upscaled.png".format(i + 1, j))

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]